In [1]:
import pandas as pd
import plotly.express as px
from collections import Counter

In [ ]:
'''
For the downloading of the datas (sql request),
we chose the most recent datas.
Besides, we are using only the ones that have score or viewcount > 0.
The reason is to obtain datas with a "good quality"
(i.e. score < 0 could be wrong tags or sentences without understandable words)
'''

In [2]:
# Load the csv files

data1 = pd.read_csv('Data_projet5\QueryResults_first_50k.csv', encoding='utf8')
data2 = pd.read_csv('Data_projet5\QueryResults_second_50k.csv', encoding='utf8')
data3 = pd.read_csv('Data_projet5\QueryResults_third_50k.csv', encoding='utf8')
data4 = pd.read_csv('Data_projet5\QueryResults_fourth_50k.csv', encoding='utf8')

In [3]:
# Concat the 4 dataFrame to obtain an unique dataFrame

data_full = pd.concat([data1, data2])
data_full = pd.concat([data_full, data3])
data_full = pd.concat([data_full, data4])
data_full = data_full.reset_index(drop=True)

In [4]:
data_full.shape

(200000, 6)

In [5]:
data_full.head()

,CreationDate,Body,Tags,Title,Score,ViewCount
0,2021-02-14 05:06:49,<p>I am a beginner in Postgres and I want to p...,<sql><postgresql>,Printing series in postgres,2,10
1,2021-02-14 03:44:31,"<p>So I forked public repo <a href=""https://gi...",<git><authentication><fork>,"Can't push to forked repo, authentication failed",1,11
2,2021-02-14 03:32:04,<p>I have a C program that I need to interface...,<c++><c><gcc><linker><clang>,"When mixing C and C++ code, does main() need t...",4,51
3,2021-02-14 02:44:35,<p>I get the following error when I run the pi...,<python-3.x><string><tuples><typeerror>,TypeError: type.__new__() argument 2 must be t...,1,38
4,2021-02-14 02:43:28,<p>Is there a way to optimize visualization ac...,<r><proc>,storing multiple AUC's and CI's in R pROC,1,11


In [6]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   CreationDate  200000 non-null  object
 1   Body          200000 non-null  object
 2   Tags          200000 non-null  object
 3   Title         200000 non-null  object
 4   Score         200000 non-null  int64 
 5   ViewCount     200000 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 9.2+ MB


In [ ]:
# The datas are using 10.7MB of the RAM.

In [7]:
# Clean the '<' and '>' for each tag

tags_full = [doc.lower().replace('<','').replace('>', ',')[:-1].split(',') for doc in data_full['Tags']]
data_full['tags_clean'] = tags_full
data_full['tags_clean'].head()

0                          [sql, postgresql]
1                [git, authentication, fork]
2               [c++, c, gcc, linker, clang]
3    [python-3.x, string, tuples, typeerror]
4                                  [r, proc]
Name: tags_clean, dtype: object

In [8]:
# Count the frequence of each tag

list_of_tags = []
for i in range(len(data_full.tags_clean)):
    list_of_tags += data_full.tags_clean[i]

counter = Counter(list_of_tags)

tmp_full = pd.DataFrame({
    'tag': list(counter.keys()),
    'freq': list(counter.values())
}).sort_values(by='freq', ascending=False).reset_index(drop=True)

In [9]:
tmp_full

,tag,freq
0,python,27574
1,javascript,18084
2,java,13997
3,android,10288
4,c#,9073
...,...,...
25014,lavfilters,1
25015,oracle-home,1
25016,capybara-webkit,1
25017,database-cleaner,1


In [46]:
fig = px.bar(
    tmp_full[:20],
    x='freq',
    y='tag',
    color='freq',
    labels={'x':'Tag', 'y':'Frequency'},
    title='Tags frequency (top 20)',
    height=550,
    width=1000
    )
fig.show()

In [16]:
# Check if each row represent a unique tag
len(tmp_full) - len(tmp_full['tag'].unique())

0

In [17]:
# Total frequency
tmp_full['freq'].sum()

647015

In [16]:
# Calculation of frequency as a percentage
tmp_full['freq_percent'] = tmp_full['freq'] / tmp_full['freq'].sum() * 100

In [17]:
tmp_full

,tag,freq,freq_percent
0,python,27574,4.261725
1,javascript,18084,2.794989
2,java,13997,2.163319
3,android,10288,1.590071
4,c#,9073,1.402286
...,...,...,...
25014,lavfilters,1,0.000155
25015,oracle-home,1,0.000155
25016,capybara-webkit,1,0.000155
25017,database-cleaner,1,0.000155


In [18]:
def number_of_tag_percent_represatation(series_of_freq, percent):

    compteur = 0
    nombre_tags = 0

    while (compteur < len(series_of_freq)):
        if(series_of_freq.cumsum()[compteur] < percent):
            compteur += 1
        else:
            nombre_tags = compteur
            #fin du while
            break

    return(nombre_tags)

In [19]:
number_of_tag_percent_represatation(tmp_full['freq_percent'], 70)

748

In [ ]:
'''
748 tags represent 70% of the most frequent tags in the dataset.
24271 tags can be ignored (they contain low informations).
'''

In [20]:
# Calculation of frequency as a percentage
# Here, we are using all the tags, which have a frequency > 1% of the frequency
# of the most frequent tag (i.e. python)
tmp_full[tmp_full['freq'] > max(tmp_full['freq'])/100]

,tag,freq,freq_percent
0,python,27574,4.261725
1,javascript,18084,2.794989
2,java,13997,2.163319
3,android,10288,1.590071
4,c#,9073,1.402286
...,...,...,...
291,ftp,282,0.043585
292,promise,282,0.043585
293,eclipse,280,0.043276
294,installation,280,0.043276


In [21]:
tmp_full[tmp_full['freq'] > max(tmp_full['freq'])/100]['freq_percent'].sum()

58.27361034906455

In [ ]:
'''
296 tags represent 58% of the most frequent tags in the dataset.
24723 tags can be ignored (they contain low informations).
'''

In [22]:
# list of the 296 tags, sorted by alphabetic order
list_296_tags = sorted(tmp_full[tmp_full['freq'] > max(tmp_full['freq'])/100]['tag'])

In [26]:
list_296_tags[:10]

['.net',
 '.net-core',
 'airflow',
 'ajax',
 'algorithm',
 'amazon-ec2',
 'amazon-s3',
 'amazon-web-services',
 'anaconda',
 'android']

In [ ]:
#---------------------#

In [27]:
# We filter the dataFrame of 200k data:
# for each row, if the tag doesn't belong to the 296 most frequent tags
# then we delete it

In [43]:
liste_index = []

for index in range(len(data_full['tags_clean'])):

    liste_of_tags = sorted(data_full['tags_clean'][index])

    compteur = 0
    tags_ok = 0

    while compteur < len(list_296_tags):
        if(list_296_tags[compteur] == liste_of_tags[tags_ok]):
            tags_ok += 1

        if(tags_ok == len(liste_of_tags)):
            liste_index.append(index)
            break

        compteur += 1

In [44]:
len(liste_index)

51427

In [45]:
data_result = data_full.iloc[liste_index].reset_index(drop=True)

In [46]:
data_result.isnull().sum()

CreationDate    0
Body            0
Tags            0
Title           0
Score           0
ViewCount       0
tags_clean      0
dtype: int64

In [50]:
data_result[['Body', 'Title', 'tags_clean']].head()

,Body,Title,tags_clean
0,<p>I am a beginner in Postgres and I want to p...,Printing series in postgres,"[sql, postgresql]"
1,<p>I have been trying to apply a git patch tha...,"Constructing a patch for either ""git am"" or ""g...",[git]
2,<p>I have a list such that:</p>\n<p><code>['DY...,Is there a python function that can sort a lis...,[python]
3,<p>My company asks every employee to work from...,Is it possible to show my GitHub contributions...,"[git, github]"
4,<p>I've spent a few hours on google searching ...,How to determine if values in a datetime colum...,"[python, pandas, numpy]"


In [52]:
data_result[['Body', 'Title', 'tags_clean']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51427 entries, 0 to 51426
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Body        51427 non-null  object
 1   Title       51427 non-null  object
 2   tags_clean  51427 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
# The new dataFrame is using 1.2 MB of RAM (10.7MB before cleaning).

In [58]:
# The new dataFrame is saved
# We will use it in order to predict the tags
data_result[['Body', 'Title', 'tags_clean']].dropna().reset_index(drop=True).to_csv('data_51k_296_tags.csv')